
Lu's code:

```python
_,embed_feat_old = model_old(inputs)
pro=np.asarray(prototype['class_mean_old'])
distance = np.average(np.sum((
    np.tile(embed_feat_old[None,:,:],[pro.shape[0],1,1])-np.tile(pro[:,None,:],[1,embed_feat_old.shape[0],1]))**2
    ,axis=2),axis=0)
W = np.exp(-distance/(2*args.sigma_train**2)) #+1e-5
W_norm = W/np.sum(W)
loss_aug = args.tradeoff * torch.sum(torch.matmul(torch.from_numpy(W_norm).cuda(),torch.sum((embed_feat-embed_feat_old).pow(2),1)))/2.
```

In [6]:
import numpy as np
import torch

torch.cuda.set_device(1)

In [17]:
EMB_DIM = 13
BATCH_SIZE = 5
PROTO_NUM = 7
TRADEOFF = 20
SIGMA =  9999.0

In [18]:
embed_feat_old = torch.randn(BATCH_SIZE, EMB_DIM).cuda()
embed_feat = torch.randn(BATCH_SIZE, EMB_DIM).cuda()
pro=torch.randn(PROTO_NUM, EMB_DIM).cuda()
embed_feat_old.shape

torch.Size([5, 13])

## Version 1

In [23]:
distance = np.average(np.sum((
    np.tile(embed_feat_old.cpu().numpy()[None,:,:],[pro.cpu().numpy().shape[0],1,1])-np.tile(pro.cpu().numpy()[:,None,:],[1,embed_feat_old.cpu().numpy().shape[0],1]))**2
    ,axis=2),axis=0)
print(distance.shape)
W = np.exp(-distance/(2*SIGMA**2)) #+1e-5
print(W)
W_norm = W/np.sum(W)
W_norm

(5,)
[0.9999999  0.99999994 0.9999999  0.99999994 0.9999999 ]


array([0.19999999, 0.2       , 0.19999999, 0.2       , 0.19999999],
      dtype=float32)

In [ ]:
print(torch.sum(torch.matmul(torch.from_numpy(W_norm).cuda(),torch.sum((embed_feat-embed_feat_old).pow(2),1)))/2.)

## Version 2

In [20]:
n = embed_feat_old.size(0)
d = embed_feat_old.size(1)
m = pro.size(0)

x = embed_feat_old.unsqueeze(1).expand(n, m, d)
y = pro.unsqueeze(0).expand(n, m, d)
dist = torch.pow(x - y, 2).sum(2)
print(dist)

W = torch.exp(-dist/(2*SIGMA**2)) #+1e-5
W_i = W.mean(1)
W_i

tensor([[29.1052, 34.8352, 24.1788, 24.3268, 41.6415, 40.5939, 24.5877],
        [19.7452, 17.5925, 18.0452, 21.9687, 19.1554, 30.2998, 10.9869],
        [30.9425, 20.7472, 30.1760, 16.4430, 25.7284, 39.9786, 24.9528],
        [17.5091, 15.6866, 18.3220, 19.3248, 20.7047, 36.8107,  8.6259],
        [26.0489, 33.0359, 25.6250, 22.1032, 34.0444, 31.8228, 16.0027]],
       device='cuda:1')


tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000], device='cuda:1')

In [21]:
WLeF_loss = (embed_feat-embed_feat_old).pow(2).sum(1).dot(W_i)
WLeF_loss

tensor(103.2415, device='cuda:1')